In [1]:
#import driver
from tf_agents.drivers import py_driver
from tf_agents.drivers.dynamic_episode_driver import DynamicEpisodeDriver
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

#import environment
from tf_agents.environments import py_environment
from tf_agents.environments import tf_py_environment #allows parallel computing for generating experiences
from tf_agents.environments.parallel_py_environment import ParallelPyEnvironment

#import replay buffer
from tf_agents import replay_buffers as rb

#import agent
from tf_agents.agents.reinforce import reinforce_agent
from tf_agents.utils import value_ops
from tf_agents.trajectories import StepType


#other used packages
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
from matplotlib.axes import Axes as ax
import numpy as np
import tensorflow as tf
import tf_agents
from tf_agents.networks import actor_distribution_network
from tf_agents.specs import array_spec
from tf_agents.utils import common
from tf_agents.trajectories import time_step as ts
from tf_agents.networks.categorical_projection_network import CategoricalProjectionNetwork
import os,gc
import pygad
from pyswarms.single.global_best import GlobalBestPSO
import time

2025-06-09 16:56:27.042366: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import environmentRL as ENV
from typing import Tuple
from tensorflow.keras import layers

In [3]:
class Agent(tf.keras.Model):
  """Combined actor-critic network."""

  def __init__(self,num_actions: int, num_hidden_units: int):
    """Initialize."""
    super().__init__()

    self.common = layers.Dense(num_hidden_units, activation="relu")
    self.actor = layers.Dense(num_actions)
    self.critic = layers.Dense(1)

  def call(self, inputs: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    x = self.common(inputs)
    return self.actor(x)

In [10]:
x0_reinforce = np.array([-1, 1],dtype=np.float32)
env=ENV.Env(x0_reinforce)
actor = Agent(2, 2)

initS=env._reset()
state = initS.observation[None, :]
reward = initS.reward 

print(f"initial state : {state}, reward: {reward}")
for i in range(10):
    action = actor(state)
    print(f"state_{i} : {state}, action_{i}: {action}")
    s=env._step(action.numpy())
    state = s.observation
    reward = s.reward
    print(f"reward:{reward}")
    # print(s)
    
    
    
    


dddd
initial state : [[-1.  1.]], reward: 0.0
state_0 : [[-1.  1.]], action_0: [[-1.0518283  0.5030183]]
 normalized_act: [[-1.        -0.9999998]]
reward:-6.5281219482421875
state_1 : [[-1.05  0.95]], action_1: [[-1.0599902  0.5069216]]
 normalized_act: [[-1.         -0.99999976]]
reward:-14.019988059997559
state_2 : [[-1.0999999  0.9      ]], action_2: [[-1.0681522  0.5108249]]
 normalized_act: [[-1.         -0.99999976]]
reward:-26.948097229003906
state_3 : [[-1.1499999   0.84999996]], action_3: [[-1.0763141  0.5147282]]
 normalized_act: [[-1.        -0.9999998]]
reward:-45.79994583129883
state_4 : [[-1.1999998   0.79999995]], action_4: [[-1.084476   0.5186315]]
 normalized_act: [[-1.         -0.99999976]]
reward:-71.07803344726562
state_5 : [[-1.2499998   0.74999994]], action_5: [[-1.0926379  0.5225348]]
 normalized_act: [[-1.         -0.99999976]]
reward:-103.29986572265625
state_6 : [[-1.2999997  0.6999999]], action_6: [[-1.1007998   0.52643806]]
 normalized_act: [[-1.         -0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define lambda_coef
lambda_coef = 0.5

# Define the function
def f(x):
    return (x - 2)**2 + lambda_coef * (x**2)

# Create x values
x = np.linspace(-5, 5, 400)
y = f(x)

# Plotting
plt.figure(figsize=(8,5))
plt.plot(x, y, label=r'$f(x) = (x-2)^2 + \lambda x^2$', color='blue')
plt.title('Objective Function Plot')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
!pip install dm-reverb==0.9.0